<a href="https://colab.research.google.com/github/aneesa-ud/UdhyamFoodZomatoData/blob/main/FoodData1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# --- Setup ---
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from google.auth import default

!pip install --upgrade -q gspread gspread-dataframe

# Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheets
sheet1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1c4DDIFfx9I_9GWRnoZTGIoDoXMl5_BnVTV76vHv7EoI")
sheet2 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1vMvPZy28AeTpR-rVNwDjflGL2Gn2Y1P5S7n27RONpxE/edit?usp=sharing")

ws1 = sheet1.get_worksheet(0)
ws2 = sheet2.get_worksheet(0)

# --- Read DataFrames ---
df1 = get_as_dataframe(ws1, evaluate_formulas=True, header=0)
df2 = get_as_dataframe(ws2, evaluate_formulas=True, header=0)

df1.dropna(how='all', inplace=True)
df2.dropna(how='all', inplace=True)

# Strip and align columns
df1.columns = df1.columns.str.strip()
df2.columns = df2.columns.str.strip()

# Convert key columns to string and strip spaces
for col in ['Restaurant ID', 'Metric']:
    df1[col] = df1[col].astype(str).str.strip()
    df2[col] = df2[col].astype(str).str.strip()

# --- Debug: Check required columns ---
required_cols = ['Restaurant ID', 'Metric', 'May']
for col in required_cols:
    if col not in df1.columns:
        raise KeyError(f"Missing column '{col}' in Sheet 1")

for col in ['Restaurant ID', 'Metric']:
    if col not in df2.columns:
        raise KeyError(f"Missing column '{col}' in Sheet 2")

# --- Create Key for Matching ---
df1['key'] = df1['Restaurant ID'] + "|" + df1['Metric']
df2['key'] = df2['Restaurant ID'] + "|" + df2['Metric']

# --- Build Lookup ---
may_lookup = df1.set_index('key')['May'].to_dict()

# --- Apply Mapping ---
df2['May'] = df2['key'].map(may_lookup)

# --- Drop Helper Column ---
df2.drop(columns=['key'], inplace=True)

# --- Debug: Print Preview ---
print("Preview of df2 with 'May' column added:")
print(df2[['Restaurant ID', 'Metric', 'May']].head())

# --- Write Back to Sheet 2 ---
set_with_dataframe(ws2, df2, include_index=False, include_column_header=True, resize=True)
print("✅ 'May' column added to Sheet 2 successfully!")


Preview of df2 with 'May' column added:
  Restaurant ID                    Metric      May
0      21075151                Sales (Rs)  2708.80
1      21075151          Delivered orders    17.00
2      21075151  Average order value (Rs)   159.34
3      21075151          Market share (%)     0.32
4      21075151            Average rating     4.00
✅ 'May' column added to Sheet 2 successfully!
